# Data Preparation

In this notebook all the datatransformation steps are excuted and described. This file should be run first to get the monthly_trajectories.csv file from the monthly_trajectories_nl dataframe. The csv file will be input for the scripts (starting with 0_Feature_Engineering.py). In the Exploratory_Data_Analysis.ipynb more details on the different dataframes can be found.

## Imports

In [1]:
# Import necessary packages
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from fuzzywuzzy import process

/Users/merelkamper/anaconda3/lib/python3.11/site-packages/fuzzywuzzy/fuzz.py:11: UserWarning: Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning
  warnings.warn('Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning')


## Data Loading

In [2]:
# Load service data
folder_path = '/Users/merelkamper/Documents/MSc Data Science/Thesis/train_rides'
files = os.listdir(folder_path)
dfs=[]

for file in files:
    if file.endswith('.csv'):
        file_path = os.path.join(folder_path, file)
        df = pd.read_csv(file_path)
        dfs.append(df)

# Combine different files                
all_train_services = pd.concat(dfs, ignore_index=True)

## Data Cleaning

In [3]:
# Clean data to NS data and trains
ns_data = all_train_services[all_train_services['Service:Company'].str.lower().str.contains('ns')]
ns_train_data = ns_data[ns_data['Service:Type'].isin(['Intercity', 'Sprinter', 'Intercity direct', 'Eurostar', 'Thalys', 'Extra trein', 
                                                      'ICE International', 'Int. Trein', 'Speciale Trein', 'Nightjet'])]    

In [11]:
print(len(all_train_services))
print(len(ns_data))

115266904
78403382


In [7]:
# RUN TRANSOFRMATION STEPS FIRST

# Filter on rides in NL
# Load station data
stations_folder = 'stations_data'
station_files = ['stations-2019-07.csv', 'stations-2020-01.csv', 'stations-2022-01.csv', 'stations-2023-09.csv']

# Initialize an empty DataFrame for station data
stations_data = pd.DataFrame()

# Read and concatenate all station files
for file in station_files:
    file_path = os.path.join(stations_folder, file)
    temp_df = pd.read_csv(file_path)
    stations_data = pd.concat([stations_data, temp_df], ignore_index=True)

# Ensure there are no duplicate station entries
stations_data = stations_data.drop_duplicates(subset=['name_long'])

# Merge final_df with stations_data to get country information for source stations
monthly_trajectories = monthly_trajectories.merge(stations_data[['name_long', 'country']], how='left', left_on='source', right_on='name_long', suffixes=('', '_source'))
monthly_trajectories = monthly_trajectories.rename(columns={'country': 'source_country'})
monthly_trajectories = monthly_trajectories.drop(columns=['name_long'])

# Merge final_df with stations_data to get country information for target stations
monthly_trajectories = monthly_trajectories.merge(stations_data[['name_long', 'country']], how='left', left_on='target', right_on='name_long', suffixes=('', '_target'))
monthly_trajectories = monthly_trajectories.rename(columns={'country': 'target_country'})
monthly_trajectories = monthly_trajectories.drop(columns=['name_long'])

# Ensure there are no duplicate columns
monthly_trajectories = monthly_trajectories.loc[:,~monthly_trajectories.columns.duplicated()]

# Filter to keep only rides where both source and target stations are in the Netherlands
monthly_trajectories_nl = monthly_trajectories[(monthly_trajectories['source_country'] == 'NL') & (monthly_trajectories['target_country'] == 'NL')]
monthly_trajectories_nl = monthly_trajectories_nl.drop(columns=['source_country','target_country'])

## Data Transformations

Transformations on the original dataset to group the trajectories per month.

In [4]:
# DATA TRANSFORMATION 1
grouped = ns_train_data.groupby('Service:RDT-ID')

rdt_ids = grouped['Service:RDT-ID'].unique()
trajectories = ns_train_data.groupby('Service:RDT-ID')['Stop:Station name'].agg(['first', 'last']).agg(' - '.join, axis=1)
dates = grouped['Service:Date'].first()
day_of_week = pd.to_datetime(dates).dt.day_name()
max_delays = grouped['Service:Maximum delay'].first()
arrival_delays_last_stop = grouped['Stop:Arrival delay'].last()
planned_stops = np.maximum(grouped.size() - 2, 0)  # Subtract 2 for departure and arrival stops
cancelled_arrivals = grouped['Stop:Arrival cancelled'].sum()
cancelled_departures = grouped['Stop:Departure cancelled'].sum()
delayed_arrivals = (ns_train_data['Stop:Arrival delay'] > 0).groupby(ns_train_data['Service:RDT-ID']).sum()
delayed_departures = (ns_train_data['Stop:Departure delay'] > 0).groupby(ns_train_data['Service:RDT-ID']).sum()
partly_cancelled = grouped['Service:Partly cancelled'].any()
completely_cancelled = grouped['Service:Completely cancelled'].all()
last_stop_cancelled = grouped.last()['Stop:Arrival cancelled']

definite_df = pd.DataFrame({
    'RDT-ID': rdt_ids,
    'Trajectory': trajectories,
    'Date': dates,
    'Day of the Week': day_of_week,
    'Maximum Delay': max_delays,
    'Arrival Delay of Last Stop (min)': arrival_delays_last_stop,
    'Nr. of Planned Stops': planned_stops,
    'Nr. of Cancelled Arrivals': cancelled_arrivals,
    'Nr. of Cancelled Departures': cancelled_departures,
    'Nr. of Delayed Arrivals': delayed_arrivals,
    'Nr. of Delayed Departures': delayed_departures,
    'Partly Cancelled': partly_cancelled,
    'Completely Cancelled': completely_cancelled,
    'Last Arrival Cancelled': last_stop_cancelled
})

# Handling of the 250 NaN values in "Arrival Delay of Last Stop (min)"" and None values in "Last Arrival Cancelled"
# Set 'Arrival Delay of Last Stop (min)' to -1 for unfinished trajectories
definite_df['Arrival Delay of Last Stop (min)'].fillna(-1, inplace=True)

# Set 'Last Arrival Cancelled' to True for these cases as they represent cancellations
definite_df['Last Arrival Cancelled'].fillna(True, inplace=True)

/var/folders/3g/j8k6m66124x5lcy646zpdz680000gn/T/ipykernel_2187/3674219698.py:38: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  definite_df['Arrival Delay of Last Stop (min)'].fillna(-1, inplace=True)
/var/folders/3g/j8k6m66124x5lcy646zpdz680000gn/T/ipykernel_2187/3674219698.py:41: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we

In [5]:
# DATA TRANSFORMATION 2
definite_df['Date'] = pd.to_datetime(definite_df['Date'])
definite_df[['source', 'target']] = definite_df['Trajectory'].str.split(' - ', expand=True)

definite_df['source'] = definite_df['source'].replace({'Amersfoort': 'Amersfoort Centraal', 'Eindhoven': 'Eindhoven Centraal'})
definite_df['target'] = definite_df['target'].replace({'Amersfoort': 'Amersfoort Centraal', 'Eindhoven': 'Eindhoven Centraal'})

rides_performed = definite_df.dropna(subset=['Arrival Delay of Last Stop (min)']).groupby(['Date','source', 'target']).size().reset_index(name='Rides planned')
delayed_arrivals = definite_df[definite_df['Arrival Delay of Last Stop (min)'] > 0.0].groupby(['Date', 'source', 'target']).size().reset_index(name='Final arrival delay')
arrival_canceled = definite_df.groupby(['Date', 'source', 'target'])['Last Arrival Cancelled'].sum().reset_index(name='Final arrival cancelled')
trajectory_canceled = definite_df.groupby(['Date', 'source', 'target'])['Completely Cancelled'].sum().reset_index(name='Completely cancelled')
intermediate_delays = definite_df[definite_df['Nr. of Delayed Arrivals'] > 0].groupby(['Date', 'source', 'target']).size().reset_index(name='Intermediate arrival delays')

trajectories_per_day = rides_performed.merge(delayed_arrivals, on=['Date', 'source', 'target'], how='left')
trajectories_per_day = trajectories_per_day.merge(arrival_canceled, on=['Date', 'source', 'target'], how='left')
trajectories_per_day = trajectories_per_day.merge(trajectory_canceled, on=['Date', 'source', 'target'], how='left')
trajectories_per_day = trajectories_per_day.merge(intermediate_delays, on=['Date', 'source', 'target'], how='left')

trajectories_per_day['Final arrival delay'].fillna(0, inplace=True)
trajectories_per_day['Final arrival delay'] = trajectories_per_day['Final arrival delay'].astype(int)
trajectories_per_day['Final arrival cancelled'] = trajectories_per_day['Final arrival cancelled'].replace(False, 0).astype(int)
trajectories_per_day['Intermediate arrival delays'] = trajectories_per_day['Intermediate arrival delays'].fillna(0).astype(int)

# Remove rows where source is the same as target
trajectories_per_day = trajectories_per_day[trajectories_per_day['source'] != trajectories_per_day['target']]

/var/folders/3g/j8k6m66124x5lcy646zpdz680000gn/T/ipykernel_2187/3437742489.py:19: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  trajectories_per_day['Final arrival delay'].fillna(0, inplace=True)


In [10]:
# DATA TRANSFORMATION 3
# Convert the Date column to datetime format and create a YearMonth column
trajectories_per_day['Date'] = pd.to_datetime(trajectories_per_day['Date'])
trajectories_per_day['YearMonth'] = trajectories_per_day['Date'].dt.to_period('M')

# Perform monthly aggregation
monthly_trajectories = trajectories_per_day.groupby(['YearMonth', 'source', 'target'], as_index=False).agg({
    'Rides planned': 'sum',
    'Final arrival delay': 'sum',
    'Final arrival cancelled': 'sum',
    'Completely cancelled': 'sum',
    'Intermediate arrival delays': 'sum'
})

# Filter out trajectories with fewer than 4 rides planned per month
min_rides_threshold = 4
print(len(monthly_trajectories))
monthly_trajectories = monthly_trajectories[monthly_trajectories['Rides planned'] >= min_rides_threshold]
print(len(monthly_trajectories))

# Recalculate the proportion delayed after monthly aggregation
monthly_trajectories['Proportion delayed'] = monthly_trajectories.apply(
    lambda row: 0 if row['Final arrival delay'] == 0 else 
    row['Final arrival delay'] / (row['Rides planned'] - row['Completely cancelled']) 
    if (row['Rides planned'] - row['Completely cancelled']) > 0 else 0, axis=1)

# Calculate the 90th percentile threshold
percentile_50 = monthly_trajectories['Proportion delayed'].quantile(0.50)

# Add a column to indicate if the delay is significant based on the threshold
monthly_trajectories['Significant Delay'] = monthly_trajectories['Proportion delayed'] > percentile_50 

39942
30416


Let's see what the threshold is for a trajectorie to be significantly delayed

In [9]:
percentile_50

0.21014492753623187

Using the 50th percentile creates a balanced dataset.

In [26]:
numberTrue = monthly_trajectories_nl[monthly_trajectories_nl['Significant Delay'] == True]
numberFalse = monthly_trajectories_nl[monthly_trajectories_nl['Significant Delay'] == False]
print(f"Number of True values in data {len(numberTrue)}, Number of False values in data {len(numberFalse)}")

Number of True values in data 14392, Number of False values in data 14165


## Output and Evaluation

In [8]:
# UNDERSTANDING TRANSFORMATIONS
print(f'The number of rows in the original dataset is {len(all_train_services)}')
print(f'The number of rows in the cleaned dataset is {len(ns_train_data)}')
print(f'The number of rows in the first transformed dataset is {len(definite_df)}')
print(f'The number of rows in the second transformed dataset is {len(trajectories_per_day)}')
print(f'The number of rows in the third transformed dataset is {len(monthly_trajectories)}')
print(f'The number of rows in the final dataset is {len(monthly_trajectories_nl)}')

The number of rows in the original dataset is 115266904
The number of rows in the cleaned dataset is 75620094
The number of rows in the first transformed dataset is 7344087
The number of rows in the second transformed dataset is 559668
The number of rows in the third transformed dataset is 30416
The number of rows in the final dataset is 28557


In [27]:
# SAVE DATASET AS CSV 
definite_df.to_csv('/Users/merelkamper/Documents/MSc Data Science/Thesis/MSc_thesis_code/data/first_transformation.csv', index=False)
trajectories_per_day.to_csv('/Users/merelkamper/Documents/MSc Data Science/Thesis/MSc_thesis_code/data/daily_trajectories.csv', index=False)
monthly_trajectories_nl.to_csv('/Users/merelkamper/Documents/MSc Data Science/Thesis/MSc_thesis_code/data/monthly_trajectories.csv', index=False)